# Exmple how to extract Entities and Relationships with LLMs, convert to an Langchain GraphDocument and put it into a AWS Neptune Graph Database via Gremlin interface

### Sources

* [https://www.aidoczh.com/langchain/v0.2/docs/integrations/graphs/azure_cosmosdb_gremlin/](https://www.aidoczh.com/langchain/v0.2/docs/integrations/graphs/azure_cosmosdb_gremlin/)
* [https://repost.aws/questions/QU1SNP6AMoQquNkPlpo5ZLVw/using-aws-neptune-with-langchain](https://repost.aws/questions/QU1SNP6AMoQquNkPlpo5ZLVw/using-aws-neptune-with-langchain)
* [https://python.langchain.com/docs/integrations/graphs/amazon_neptune_open_cypher/](https://python.langchain.com/docs/integrations/graphs/amazon_neptune_open_cypher/)
* [https://python.langchain.com/docs/integrations/graphs/amazon_neptune_open_cypher/](https://python.langchain.com/docs/integrations/graphs/amazon_neptune_open_cypher/)
* [https://python.langchain.com/docs/integrations/chat/bedrock/](https://python.langchain.com/docs/integrations/chat/bedrock/)



## Test the Neptune DB status 

In [121]:
!curl https://db-neptune-1.cluster-ro-cqniyw0gi5se.us-east-1.neptune.amazonaws.com:8182/openCypher/status

{
    "acceptedQueryCount" : 0,
    "runningQueryCount" : 0,
    "queries" : [ ]
}

## Get Neptune DB status and configuration

In [122]:
%status

UsageError: Line magic function `%status` not found.


In [123]:
%graph_notebook_config

UsageError: Line magic function `%graph_notebook_config` not found.


## Install necessary libraries

In [124]:
%pip install -qU langchain-aws langchain_community langchain_experimental langchain tiktoken boto3 json-repair langchain-neo4j

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
aiobotocore 2.21.1 requires botocore<1.37.2,>=1.37.0, but you have botocore 1.38.27 which is incompatible.
awscli 1.40.0 requires botocore==1.38.1, but you have botocore 1.38.27 which is incompatible.
awscli 1.40.0 requires s3transfer<0.13.0,>=0.12.0, but you have s3transfer 0.13.0 which is incompatible.
graph-notebook 5.0.1 requires json-repair==0.29.2, but you have json-repair 0.39.1 which is incompatible.
graph-notebook 5.0.1 requires neo4j<=5.23.1,>=5.0.0, but you have neo4j 5.28.1 which is incompatible.
graph-notebook 5.0.1 requires numpy<1.24.0,>=1.23.5, but you have numpy 1.26.4 which is incompatible.
s3fs 2025.3.2 requires fsspec==2025.3.2.*, but you have fsspec 2024.12.0 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [145]:
%pip install -qU nest_asyncio unstructured markdown

Note: you may need to restart the kernel to use updated packages.


## Import necessary libraries

In [126]:
import nest_asyncio
nest_asyncio.apply()

In [143]:
import json
import boto3
import os
import glob
import sys
import pprint
import numpy as np
from getpass import getpass
from langchain_aws import BedrockLLM
from langchain_aws import BedrockEmbeddings
from langchain_aws import ChatBedrockConverse
from langchain_aws.graphs import NeptuneGraph
from langchain_aws.graphs import NeptuneAnalyticsGraph
from langchain_aws.chains import create_neptune_opencypher_qa_chain
from langchain_experimental.graph_transformers import LLMGraphTransformer
from langchain_community.graphs.graph_document import GraphDocument
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.documents import Document
from langchain_community.document_loaders import UnstructuredMarkdownLoader
from langchain_community.callbacks.bedrock_anthropic_callback import BedrockAnthropicTokenUsageCallbackHandler
from gremlin_python.driver import client, serializer
from gremlin_python.process.traversal import T

In [128]:
MODELS = {
    "Claude 3.7 Sonnet": "us.anthropic.claude-3-7-sonnet-20250219-v1:0",
    "Claude 3.5 Sonnet": "us.anthropic.claude-3-5-sonnet-20240620-v1:0",
    "Claude 3.5 Haiku": "us.anthropic.claude-3-5-haiku-20241022-v1:0",
    "Amazon Nova Pro": "us.amazon.nova-pro-v1:0",
    "Amazon Nova Micro": "us.amazon.nova-micro-v1:0",
    "DeepSeek-R1": "us.deepseek.r1-v1:0",
    "Meta Llama 3.1 70B Instruct": "us.meta.llama3-1-70b-instruct-v1:0"
}

## Enter AWS credentials

In [129]:
default_region = "us-east-1"
AWS_ACCESS_KEY = getpass("AWS Acces key: ")
AWS_SECRET_KEY = getpass("AWS Secret key: ")
AWS_REGION = input(f"AWS Region [default: {default_region}]: ") or default_region

AWS Acces key:  ········
AWS Secret key:  ········
AWS Region [default: us-east-1]:  


## Initialize the AWS clients

In [130]:
bedrock_client = boto3.client(
    service_name="bedrock-runtime",
    region_name=AWS_REGION,
    aws_access_key_id=AWS_ACCESS_KEY,
    aws_secret_access_key=AWS_SECRET_KEY,
)

In [131]:
neptune_client = boto3.client(
    service_name="neptune-graph",
    region_name=AWS_REGION,
    aws_access_key_id=AWS_ACCESS_KEY,
    aws_secret_access_key=AWS_SECRET_KEY,
    verify=False,
)

## Initialize the LLM

In [132]:
default_model_id = "us.anthropic.claude-3-7-sonnet-20250219-v1:0"
AWS_MODEL_ID = input(f"AWS model [default: {default_model_id}]: ") or default_model_id

AWS model [default: us.anthropic.claude-3-7-sonnet-20250219-v1:0]:  


In [133]:
llm = ChatBedrockConverse(
    client=bedrock_client,
    model=AWS_MODEL_ID,
    temperature=0,
)

## Text

In [14]:
transcript = """
hey everybody it's charlie from daily motor and if you just purchased a 2024 volkswagen id4 or maybe a little bit newer one if 
you're watching this after the fact or maybe you're just testing one out this video is for you we're going to show you how to 
use the 12.9 inch infotainment screen we're going to show you how to pair up a phone use apple carplay and android auto we're 
going to test out the voice commands and try to answer any questions you might have now if you do want to see more on the car 
check the link below we've got a dedicated review of the seven speaker sound system we're going to be doing a full review i 
don't think we're going to do a range test on this model but we have range test some range tested some previous id4s so feel 
free to find that below as well so oh my gosh all right a little bit of a sneak peek here we've got a wake word that has come 
awake on me a few times here and not super convenient but in front of you you see a pretty straightforward gauge cluster screen 
really not too much to tell you other than that it tells you your digital speed speed limit your cruise control your charge 
percentage as well as a mileage and the gear that you're in maybe some information there about seat belts i actually really 
like how straightforward this is you don't need any fancy power meter it tells you both your percentage of battery and your 
distance to empty your kind of mileage they're estimated just a nice easy to read digital speed readout the resolution's nice 
this is a very good compromise i see to having a simple clean layout but still giving you something to see right in front of 
you no sort of head-up display so let's move right on into the center screen this is an updated system for the volkswagen id4 
and as i understand not all models are getting it some of the more basic id4s are sticking with the 12-inch screen now i don't 
know for sure if that's running the previous operating system that version or if it's this version on the smaller screen i 
think it's the older version of the operating system though but this one if we come to the home screen this is kind of what 
you're going to see your basic home screen here you've got some persistent buttons up on the top for various functionality of 
the system this is essentially an app screen if you will we'll get back to that here in a moment you've got the outside 
temperature a quick toggle to your android auto or apple carplay and some information over here on the right such as time and 
signal strength for the phone and along the bottom you have some persistent climate controls heated and ventilated seats and 
temperature to bring up the full climate control screen you click climate and then you can see all of your controls for the 
climate you can also just change the temperature right here with these controls and then right here in the center you've got 
volume and a little on and off button for turning the media on and off on this home screen you're seeing about two-thirds of it 
on the left taken up by the navigation and then on the right looks like some smart kind of things you might want to click right 
away kind of quick toggles if you will and then some media controls and information about charging let's get back to this home 
screen you always get back home with that home button if we slide to the right we see some more tiles that give us information 
looks like we've got phone vehicle status and some tips and tricks and you can customize the tiles on our home screen by let's 
see right here the layout can move things around and there's there's a we're not going to go into every single little detail 
and how to on the system this is more a general overview and then it'll get you to the proper screens to be able to spend more 
time playing around with them but man there is a lot going on here and this is this is one of my first interactions with the 
system as well so i'm learning it to display area operating menu okay this is telling you the things i just told you operation 
zoom in and out yeah that's that's we're familiar with how to do that customization customize the tiles on the home screen do 
this press and hold the tile list of various functions and then a function select a function to be displayed menu sorting order 
arrange the functions all right so that's on the main kind of menu deal so if we go home and we want this one we can customize 
it right here then this comes up and you can choose the different things maybe i wanted uh sound adjustments to be right there 
and i can then have that on the main screen and click that it's bringing me right to my sound settings one thing i do 
appreciate about the screen is how quickly it responds to to inputs there's not really that micro lag that you get with a lot 
of screens on the market these days okay so you can even swap out what's on this big center area screen as well so say for 
example on this one press and hold if i wanted that one to be driving data and there you go it's driving data so you really 
ought to spend time like this play around with the car and customize it set up the main screens for what you're going to be 
using most often let's go on along this top menu here we see a vehicle quick toggle screen so little uh commands or 
functionality that you might want to get to quickly like opening the roof really roof settings oh you can close close the 
sunshade from there that's interesting i didn't know you could do that from the infotainment system go in back and change your 
drive modes light settings etc in the parking screen right here that should give you the park assist or the ability to bring 
your rear-facing camera up some of the higher end models might have a 360 cam in the assist screen this is an easy way to turn 
on or off your different driver assistant functionality functionality up here that's quick toggle for your drive modes eco eco 
custom sport and comfort coming back home then we've got a kind of charging and battery information screen optimize certain 
things then coming back home you can actually swipe down right here to get to a quick toggle screen they should be able to 
access from pretty much any menu that gives you the ability to change the brightness of the screen especially at night and 
change between the bright and dark themes so now around the bright theme it'll look a little different you can adjust things 
like your sound settings navigate to your last destination volkswagen is really trying to give you a smart system here and and 
for some people that's going to be frustrating to use if you don't really get comfortable with it and figure out a way to make 
it work best for you and for your lifestyle but the more comfortable you are with it the more you're going to like it so let's 
bring up all of our apps now if we go to telephone you can see any sort of functionality that you might be able might want to 
do if you sync your contacts you can get those there make calls dial in a number etc going back let's go to radio and media 
radio and media you can see go to radio if i want to go let's see what's been last listened to oh no listening history all 
right news channel news and public radio let's go cnbc and if we back that out does that actually give us kind of a now playing 
screen player yeah there you go you can see your sirius xm you can pause go back change channel etc right back to live and 
favorite the channel coming back home let's go to navigation you can see your big map taking up a decent amount of the screen 
you can move around pinch to zoom fairly easily change which direction you're looking at the map come on should be able to i 
don't know why that's not working right now strange there we go i guess it just didn't want to be zoomed out or zoomed in and 
you can put in destinations we'll do a little bit more of that when we use the voice commands later under the vehicle screen we 
can you get it sort of broken up between interior and exterior so you can adjust functionality for the exterior lights how long 
you want them to stay on a little animations etc we can take a look at our tires see what our tire pressures are set at adjust 
those if you got winter tires on here and you want a speed warning for not going too fast on you can set that their headlights 
got the dynamic cornering lights so as you turn you get a little bit of extra lighting on their mirrors do you want them to 
drop down and and look down at the curb as you're in reverse you can do that for interior if we go to cockpit this gives us the 
ability to reset the driving data every time we start the vehicle or at different intervals id light this is your interior 
ambient lighting you can adjust the brightness on that you can see it moving around up there as well as the basic functionality 
so that id light is actually this light right here in front again let me adjust the brightness and you can see it right there 
it'll do things like when you're charging it'll shine a light and show you how much you're charging if you get an error or 
something's wrong with your vehicle it'll pulse at you for navigation you can have it light up for a turn driving and 
maneuvering tells you to take over the steering when you got the pilot assist going it's uh yeah it's kind of a neat 
functionality there interior light this is the ability to dim or brighten the lights because on this id4 for 2024 we've got 
some back lighting for the volume and climate controls under seats easy entry do you want the seat to move backward as you get 
in and out of it and turn on or off the vehicle and roof again that ability to open or close the roof from there then on the 
same vehicle screen we can see some status information this is where no it's still not letting us actually see our tire 
pressures it's just giving us a thumbs up showing us our odometer and the tripometer there as well as telling us we're good on 
service for now the ability to put your wipers in service position that's important if you want to change your wipers out 
yourself down under data we can see how efficient we've been driving since the start since it's been reset or since we last 
charged the vehicle pretty nice and straightforward there coming back to the app screen we'll skip over android auto for now we 
saw parking and charging auxiliary climate this is going to be if you want the climate control to stay on after you get out of 
the vehicle or if you want it to turn on at a certain time i actually took advantage of this yesterday i knew i was going to be 
getting in the vehicle after it was parked outside around 5 p.m so i toggled it to turn on and set the climate to 72 by 5 p.m i 
got in the car was nice and cool electric car advantages there let's skip settings for now we'll come back we saw drive mode 
here's your ambient lighting i don't know why it wasn't in the other setting screen maybe i missed over it but there are some 
preset ones you can choose from here or you hit custom and go down to custom here and you can change and customize your ambient 
lighting as a two-tone setup for the different parts of the vehicle and you can adjust the brightness for the different 
settings or parts of the vehicle as well so a lot of customization i like that volkswagen's doing that i mean they are leds so 
why shouldn't they be customizable and you can also just turn them all off right there if you want actually get a little bit 
more of a purple going here that usually looks good on camera we're going to skip over seats and sound we'll go to users if you 
have multiple people in the household that drive the car then you might want to set up different users for them save the radio 
presets climate presets and i think probably some seat settings as well although i do see buttons over here for seat memory so 
perhaps the touch screen can't operate that i can't exactly let you know coming back in we're going to skip over ida for now 
we'll get to that here in a moment you have the ability to see tips and help right there um we've got a climate control screen 
where i saw that and so that's about all you need from there so let's get into settings see anything we missed under screen you 
can turn the screen tone on or off we're going to get to that here in a moment i think that's under sound actually i know right 
here infotainment system touch screen tone so if you want to hear those little beeps as you touch the touch the screen you're 
going to want to have that on but i like that off time and date if you want to manually set your time you can do that from 
their language what do we have in here english spanish and french no german interestingly enough the ability to customize which 
units you see things in voice assistant settings what we got in here news feed okay dialogue tone the start and end dialogue oh 
to make a little beep sound okay all right that's great you can adjust the volume there there display the voice input probably 
want that as well answer calls without the wake word okay this is an important option right here if you do plan on selling your 
id4 you're going to want to restore the factory settings of the vehicle to clear everything out for you maybe you just bought 
an id4 you want to clear it out as well under network you have the ability to connect the system to a a wi-fi network to 
download updates and etc and you can use the infotainment system as a hotspot as well if you pay for it monthly power devices 
in the cabin such as tablets or laptops under mobile devices this is going to be how you pair or unpair different devices maybe 
delete them from the car we'll get to that later on information nothing really important there and then you actually have a 
configuration wizard so this is going to be if you first purchase the car you want to go in here and it'll set up a lot of 
things for you that's nifty to have that right there all right let's dive into the voice commands now but before we actually 
start them out let's go to this IDA screen so that's the name of the ID assistant it'll show you different suggestions on 
things you can see and or things you can say red is definitely my color what is that based on your current consumption you will 
still be able to drive 215 miles oh okay well so those are things you can actually click on to see there are a few different 
ways to activate it if we turn this off then we can say hello IDA or hello ID with that on and that's going to actually trigger 
the system or you can create your custom wake word so if I do something like right here tips for creating a wake word three or 
more syllables unique and not easily confused to be clearly and concisely set as you normally would hey everybody okay so now 
if I go hey everybody then it's it activates and and I can do some voice commands so we're obviously going to delete that 
because we wouldn't want that activating every time I start a video here in the car but let's get into it right now let's do 
something like hello ID navigate to the nearest Starbucks oh I still have to learn about that what that should be very 
straightforward here is an overview of the functions that you can operate by voice please pick one by saying the line or the 
function name okay let's let's just okay let's let's just cancel continue press here or activate idea go go home all right 
let's try this again you can also start the voice command with the steering wheel by pressing this button find the nearest 
Starbucks I'm still puzzled why that should be really straightforward what about a charger where's the nearest charging station 
search for charging station in the vicinity please select a POI from the list okay so it looks like it's picking some charge 
point Electrify America stations let's try search for Starbucks I'm still puzzled man so it can't do Starbucks all right um 
obviously you could just put in a destination like navigate to 1234 Main Street Los Angeles California did you mean 1234 South 
Main Street Los Angeles yes I found multiple alternatives for 1234 please say the line number two line two okay 1236 South Main 
Street Los Angeles California drive safely so that's a pretty a pretty pleasant system I'm just surprised it can't do the 
Starbucks thing that all the other cars can do but overall it is working pretty well and uh all right let's just try canceling 
hey hello ID hello ID cancel navigation stopping route guidance hello ID set the temperature to 62 degrees all right I'm 
setting the temperature to 62.0 degrees 62.0 great hello ID what's the weather like tomorrow I'm still puzzled all right so I 
can't do weather um let's try one more tell me a joke what has four legs and can fly two birds two birds that's pretty funny I 
like that all right so I when it does work I like the way the system works it responds well it's I like seeing the words up 
there it's got a pleasant sound to it it's just a few functionalities there I'm surprised it can't do so uh now is the time we 
dive into Android Auto but it seems to have disconnected my Android Auto so we're gonna go into settings and mobile devices try 
reconnecting this oh it's turn so turning off the wi-fi hotspot seems to have uh turned off the Android Auto okay there we go 
Android Auto coming up wirelessly taking up most of the screen but we still have our hot toggle screens on the top and bottom 
nice split screen format here with the google maps on the left and YouTube music on the right now interestingly it's in the 
nighttime setting and it must be because we have the screen on the dark setting which is fine that's that's how I'd like it but 
most cars don't do it that way here's our apps let's go down and bring up YouTube music hit the super mix see what we're 
getting some corn okay so we got that and if we go back home then it gives us our music on the right nav on the left let's 
bring up google maps full screen this is how most people are probably going to be using navigation anyway look at that 
responsiveness works really well very nice refresh pinch to zoom looking good now let's head on home and we'll connect our 
iPhone we got the iPhone right here we're going to bring it into the settings screen go down to bluetooth and then the car I'm 
going to bring up our app screen go to settings network mobile devices and any moment here it should pop up with the iPhone 
baby oh well down here on the iPhone I see there we go Charlie's iPhone we're going to click right here to get Apple CarPlay to 
come up and then anything that pops up on the phone or vehicle screen I'm going to accept pair and there we have it Apple 
CarPlay coming up taking up the whole screen definitely a larger resolution than the Android Auto so things look a little 
chunkier for better for worse but we still have that split screen setup with navigation and media let's bring up YouTube music 
first again and once it loads we'll hit super mix see what we're getting on the iPhone baby no money all right so two different 
artists here and then we're usually getting coming back home then you see you have your media on the right now on the left this 
is what your app screen is going to look like again really good refresh I'm overall really happy with the screen let's go to 
our Google Maps bring it up full screen again looking good moving around cannot pinch to zoom and it does look like our refresh 
is a little lower on the iPhone as well but all things considered working nicely and bring it back home right there so there we 
go 12.9 inch touchscreen here on the Volkswagen ID4 for 2024 I like it decently I think I like it better than the older system 
it responds really really well I like the quality of the screen the visuals like I said a few different little quirks there but 
overall a system that deserves to be spent time with and played around with so that you learn how to use it properly if there 
anything else I can answer try to ask it below and I'll try to get back to you and we'll see on the next one I'm Charlie from 
Daily Motor and as always, Drive On!
"""

In [137]:
md_files = glob.glob("./*.md", recursive=False)

In [138]:
md_files

['./1be03369cce8fde3db027342142350a79b2ea419fa7afc11534325701afdd771.md',
 './e4d4afb7254a62bd92ce03a57bd76aa27c613d7852536df29f6b594208725a88.md',
 './681bb5718154ff93bf5d85262665d92cbd3805a85c79670aa85ab6a4cc1ac707.md',
 './5527f0d44ad115b1cf8de07fe602658e895a220829e7051e21dd710cf65743e9.md',
 './b56b1cef43f3ac6bdb75ff047c40fac14fb1ea5e2d2ff3bb1cbc62a713b6e0b4.md',
 './8d1ecc800380bdafb087195011aa7de9b31e780c59e03f7163f82bd8f1f36dfa.md']

In [148]:
documents = []
for md_file in md_files:
    loader = UnstructuredMarkdownLoader(md_file)
    document = loader.load()
    documents.append(document)

In [150]:
llm_transformer = LLMGraphTransformer(llm=llm)

In [158]:
graphs = []
for doc in documents:
    graph = await llm_transformer.aconvert_to_graph_documents(doc)
    graphs.append(graph)

In [ ]:
host = "db-neptune-1.cluster-cqniyw0gi5se.us-east-1.neptune.amazonaws.com"
port = 8182
use_https = True

neptune_client = client.Client(
    f'wss://{host}:{port}/gremlin',
    'g',
    username='',  # If IAM is enabled, use AWS credentials
    password='',  # If IAM is enabled, use AWS credentials
    message_serializer=serializer.GraphSONSerializersV2d0()
)

In [160]:
host = "db-neptune-1.cluster-cqniyw0gi5se.us-east-1.neptune.amazonaws.com"
port = 8182
use_https = True

neptune_client = client.Client(
    f'wss://{host}:{port}/gremlin',
    'g',
    username='',  # If IAM is enabled, use AWS credentials
    password='',  # If IAM is enabled, use AWS credentials
    message_serializer=serializer.GraphSONSerializersV2d0()
)

In [161]:
def delete_all_edges():
    query = "g.E().drop()"
    result = neptune_client.submitAsync(query).result()
    result.all().result()

def delete_all_nodes():
    query = "g.V().drop()"
    result = neptune_client.submitAsync(query).result()
    result.all().result()

def delete_entire_graph():
    delete_all_edges()
    delete_all_nodes()
    
def add_node(node):
    query = f"g.addV('{node['type']}').property(T.id, '{node['id']}')"
    for key, value in node['properties'].items():
        query += f".property('{key}', '{value}')"
    result = neptune_client.submitAsync(query).result()
    return result

def add_edge(edge):
    source_id = edge["source"]
    target_id = edge["target"]
    edge_type = edge["type"]

    try:
        query = f"g.V('{source_id}').as('a').V('{target_id}').addE('{edge_type}').from('a')"
        for key, value in edge["properties"].items():
            query += f".property('{key}', '{value}')"
        
        result = neptune_client.submitAsync(query).result()
        result.all().result()
        print(f"✅ Edge created: {source_id} -> {target_id}")
    except Exception as e:
        print(f"❌ ERROR creating edge {source_id} -> {target_id}: {e}")

# Schritt 3: Gesamten Graph einfügen
def insert_graph(graph_doc: GraphDocument):
    for node in graph_doc.nodes:
        gremlin_node = {
            "id": node.id,
            "type": node.type,
            "properties": node.properties or {}
        }
        add_node(gremlin_node)

    for edge in graph_doc.relationships:
        gremlin_edge = {
            "source": edge.source.id,
            "target": edge.target.id,
            "type": edge.type,
            "properties": edge.properties or {}
        }
        add_edge(gremlin_edge)

In [162]:
delete_entire_graph()


Received error message '{'requestId': '5b0aa07f-7f5c-4652-9c9a-a431dc6b1dbf', 'status': {'message': '{"detailedMessage":"Vertex with id already exists: ","code":"ConstraintViolationException","requestId":"5b0aa07f-7f5c-4652-9c9a-a431dc6b1dbf","message":"Vertex with id already exists: "}', 'code': 499, 'attributes': {}}, 'result': {'data': None, 'meta': {}}}'

With results dictionary '{'5b0aa07f-7f5c-4652-9c9a-a431dc6b1dbf': <gremlin_python.driver.resultset.ResultSet object at 0x7fc923f3f5b0>}'

Received error message '{'requestId': '10a621d7-9495-455b-b0b7-69224d24d731', 'status': {'message': '{"detailedMessage":"Vertex with id already exists: ","code":"ConstraintViolationException","requestId":"10a621d7-9495-455b-b0b7-69224d24d731","message":"Vertex with id already exists: "}', 'code': 499, 'attributes': {}}, 'result': {'data': None, 'meta': {}}}'

With results dictionary '{'10a621d7-9495-455b-b0b7-69224d24d731': <gremlin_python.driver.resultset.ResultSet object at 0x7fc923f44fd0>}'


In [ ]:
for g in graphs:
    insert_graph(g[0])

## Setup the AWS Neptune database connection

In [15]:
llm_transformer = LLMGraphTransformer(llm=llm)

In [16]:
documents = [Document(page_content=transcript)]

In [17]:
data = await llm_transformer.aconvert_to_graph_documents(documents)

In [90]:
host = "db-neptune-1.cluster-cqniyw0gi5se.us-east-1.neptune.amazonaws.com"
port = 8182
use_https = True

In [91]:
neptune_client = client.Client(
    f'wss://{host}:{port}/gremlin',
    'g',
    username='',  # If IAM is enabled, use AWS credentials
    password='',  # If IAM is enabled, use AWS credentials
    message_serializer=serializer.GraphSONSerializersV2d0()
)

In [92]:
def delete_all_edges():
    query = "g.E().drop()"
    result = neptune_client.submitAsync(query).result()
    result.all().result()

def delete_all_nodes():
    query = "g.V().drop()"
    result = neptune_client.submitAsync(query).result()
    result.all().result()

def delete_entire_graph():
    delete_all_edges()
    delete_all_nodes()
    
def add_node(node):
    query = f"g.addV('{node['type']}').property(T.id, '{node['id']}')"
    for key, value in node['properties'].items():
        query += f".property('{key}', '{value}')"
    result = neptune_client.submitAsync(query).result()
    return result

def add_edge(edge):
    source_id = edge["source"]
    target_id = edge["target"]
    edge_type = edge["type"]

    try:
        query = f"g.V('{source_id}').as('a').V('{target_id}').addE('{edge_type}').from('a')"
        for key, value in edge["properties"].items():
            query += f".property('{key}', '{value}')"
        
        result = neptune_client.submitAsync(query).result()
        result.all().result()
        print(f"✅ Edge created: {source_id} -> {target_id}")
    except Exception as e:
        print(f"❌ ERROR creating edge {source_id} -> {target_id}: {e}")

# Schritt 3: Gesamten Graph einfügen
def insert_graph(graph_doc: GraphDocument):
    for node in graph_doc.nodes:
        gremlin_node = {
            "id": node.id,
            "type": node.type,
            "properties": node.properties or {}
        }
        add_node(gremlin_node)

    for edge in graph_doc.relationships:
        gremlin_edge = {
            "source": edge.source.id,
            "target": edge.target.id,
            "type": edge.type,
            "properties": edge.properties or {}
        }
        add_edge(gremlin_edge)

In [93]:
delete_entire_graph()

In [94]:
insert_graph(data[0])

✅ Edge created: Charlie -> Daily Motor
✅ Edge created: Charlie -> 2024 Volkswagen Id4
✅ Edge created: 2024 Volkswagen Id4 -> 12.9 Inch Infotainment Screen
✅ Edge created: 2024 Volkswagen Id4 -> Gauge Cluster Screen
✅ Edge created: 12.9 Inch Infotainment Screen -> Android Auto
✅ Edge created: 12.9 Inch Infotainment Screen -> Apple Carplay
✅ Edge created: 12.9 Inch Infotainment Screen -> Voice Commands
✅ Edge created: 12.9 Inch Infotainment Screen -> Ida
✅ Edge created: Ida -> Voice Commands
✅ Edge created: 12.9 Inch Infotainment Screen -> Navigation System
✅ Edge created: 12.9 Inch Infotainment Screen -> Climate Control
✅ Edge created: 12.9 Inch Infotainment Screen -> Ambient Lighting
✅ Edge created: 12.9 Inch Infotainment Screen -> Bluetooth
✅ Edge created: 12.9 Inch Infotainment Screen -> Wi-Fi
✅ Edge created: Android Auto -> Google Maps
✅ Edge created: Android Auto -> Youtube Music
✅ Edge created: Apple Carplay -> Google Maps
✅ Edge created: Apple Carplay -> Youtube Music
✅ Edge crea

## Langchain QA

In [164]:
from langchain_community.graphs import GremlinGraph
from langchain_community.graphs.graph_document import GraphDocument, Node, Relationship
from langchain_core.documents import Document
from langchain_community.chains.graph_qa.gremlin import GremlinQAChain

In [165]:
graph = GremlinGraph(
    url=f"wss://{host}:{port}/gremlin",
    username='',
    password='',
)

In [166]:
#graph.add_graph_documents(data[100:])

In [167]:
graph.refresh_schema()

In [168]:
print(graph.schema)

Vertex labels are the following:
Organization,Company,Vehicle,Car,Technology,Location,Person,Feature,Product,Website,Vehicle category,Component,Specification,Event
Edge labels are the following:
COMPETES_WITH,VISITED,HAS,WORKS_FOR,EXPERIENCED_FAILURE_WITH,HAS_OPTION,MANUFACTURED_BY,MANUFACTURES,OWNS,LOCATED_IN,SOLD_IN,TESTED_AT,HAS_VARIANT,REVIEWS,COMPATIBLE_WITH,STARTED_TRIP_FROM,DEVELOPS,EQUIPPED_WITH,FITTED_WITH,TRANSPORTED,IS_VERSION_OF,TRAVELED_TO,FEATURES,USES,STANDARD_FOR,TESTED,OPTIONAL_FOR,REVIEWED,IS_A,TEST_DROVE,PUBLISHED_ARTICLE_ABOUT,PART_OF,MENTIONED_DELAY,SERVICED_AT,PLANS_TO_BUY,DEPARTED_FROM,ABOUT,OUTPERFORMED_IN_WINTER_RANGE,BASED_ON,CRITICIZED_INTERIOR_SIZE,VARIANT_OF,CO_DEVELOPS,WROTE_ABOUT,DEVELOPED_BY,FOUNDED,FEATURED_IN,TESTED_IN,WORKS_AT,HAS_NAME,CONSIDERING_BUYING
Vertices have following properties:
{}


In [169]:
chain = GremlinQAChain.from_llm(
    llm=llm,
    graph=graph,
    verbose=True,
    allow_dangerous_requests=True
)

In [185]:
chain.invoke("What about Compa")




> Entering new GremlinQAChain chain...
Generated gremlin:
g.V().hasLabel('Car')
Full Context:
[v[Audi Q6 E-Tron], v[Polestar 3], v[Tesla Model Y], v[Bmw Ix3], v[Macan Base Model], v[Taycan], v[Polestar 4], v[Macan 4], v[A6 E-Tron], v[Kia Ev6], v[Macan], v[Q6 E-Tron], v[Porsche Macan], v[Lotus Eletre], v[Tesla Model X], v[Porsche Taycan], v[Macan 4S], v[Maserati Grecale Folgore], v[Macan Turbo], v[Ford Mustang Mach-E], v[Porsche Macan Electric]]

> Finished chain.


{'query': 'What about Cars',
 'result': 'Based on the information provided, there are several electric and hybrid cars available including Audi Q6 E-Tron, Polestar 3, Tesla Model Y, BMW iX3, various Porsche models (Macan Base Model, Taycan, Macan 4, Macan 4S, Macan Turbo, Porsche Macan Electric), A6 E-Tron, Kia EV6, Lotus Eletre, Tesla Model X, Maserati Grecale Folgore, and Ford Mustang Mach-E. These represent a range of electric vehicles from different manufacturers across various price points and styles.'}